### Clone Repo:

In [ ]:
!git clone https://github.com/jhell1717/1d_heat_gpu.git

%cd 1d_heat_gpu/

### Install Dependencies:

In [ ]:
!pip install -q numba numpy matplotlib

### Import Modules from Cloned Repo:

In [ ]:
import sys
import time
sys.path.append("/content/1d_heat_gpu")

from heat1d.grid import Grid1D
from heat1d.physics import HeatEquationParams
from heat1d.bc import DirichletBC
from heat1d.ic import gaussian_hotspot
from heat1d.simulation import HeatSimulation, SimulationConfig
from heat1d.utils import cuda_available
from heat1d.viz import plot_space_time

import numpy as np

### Simulation Setup:

In [ ]:
L = 1.0
N = 10000          # Colab GPU can handle this easily
alpha = 0.1        # exaggerated for visualization

grid = Grid1D(L=L, N=N)
params = HeatEquationParams(alpha=alpha, r=0.45)
bc = DirichletBC(left=0.0, right=0.0)

sim = HeatSimulation(grid, params, bc)

x = grid.x
T0 = gaussian_hotspot(x, base=0.0, amp=1.0, x0=0.5 * L, sigma=0.05 * L)

cfg = SimulationConfig(t_final=0.01, snapshot_every=100)

### Run CPU:

In [ ]:
start = time.time()
t_cpu, frames_cpu = sim.run_cpu(T0, cfg)
end = time.time()

print(f'CPU Simulation took: {end-start} seconds.')

In [ ]:
if cuda_available():
    start = time.time()
    t_gpu, frames_gpu = sim.run_gpu(T0, cfg)
    end = time.time()

    print(f'GPU Simulation took {end-start} seconds.')
    # Sanity check: compare frames (same snapshot schedule)
    if len(t_cpu) == len(t_gpu):
        err = np.max(np.abs(frames_cpu - frames_gpu))
        print(f"Max |CPU-GPU| over snapshots: {err:.3e}")

    plot_space_time(x, t_gpu, frames_gpu, title="GPU: 1D Heat Equation (space-time)")
    # animate_line(x, t_gpu, frames_gpu, title="GPU: 1D Heat Equation")


else:
    print("CUDA not available; skipping GPU run.")